In [165]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime

In [166]:
cwd = os.path.join(os.getcwd(), 'Group Coursework Brief-20221106', 'Data_Files', 'Data_Files')
dirName_trainData = os.path.join(cwd, 'epl-full-training.csv')

In [167]:
df_epl_train = pd.read_csv(dirName_trainData)
cols = ["Div","Date","HomeTeam","AwayTeam","FTHG", "FTAG","FTR","HTHG","HTAG","HTR","Referee","HS","AS", "HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"]

df_epl_train = df_epl_train.loc[:, cols]
df_epl_train = df_epl_train.reset_index(drop=True)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_epl_train.head())

C:\Users\filip\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (98) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,17/08/2002,Blackburn,Sunderland,0.0,0.0,D,0.0,0.0,D,D Elleray,15.0,7.0,5.0,3.0,14.0,11.0,9.0,1.0,1.0,2.0,0.0,0.0
1,E0,17/08/2002,Charlton,Chelsea,2.0,3.0,A,2.0,1.0,H,G Barber,5.0,21.0,5.0,12.0,10.0,12.0,3.0,6.0,0.0,3.0,1.0,0.0
2,E0,17/08/2002,Everton,Tottenham,2.0,2.0,D,1.0,0.0,H,N Barry,13.0,10.0,9.0,5.0,18.0,4.0,10.0,5.0,1.0,1.0,0.0,0.0
3,E0,17/08/2002,Fulham,Bolton,4.0,1.0,H,3.0,1.0,H,A Wiley,13.0,3.0,6.0,1.0,16.0,12.0,7.0,4.0,1.0,2.0,0.0,0.0
4,E0,17/08/2002,Leeds,Man City,3.0,0.0,H,2.0,0.0,H,G Poll,13.0,18.0,8.0,10.0,13.0,13.0,2.0,7.0,1.0,1.0,0.0,0.0


In [168]:
# Transform the date column from strings into datetime objects
df_epl_train["Date"] = pd.to_datetime(df_epl_train["Date"], dayfirst=True)

In [169]:
df_epl_train.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,2002-08-17,Blackburn,Sunderland,0.0,0.0,D,0.0,0.0,D,...,5.0,3.0,14.0,11.0,9.0,1.0,1.0,2.0,0.0,0.0
1,E0,2002-08-17,Charlton,Chelsea,2.0,3.0,A,2.0,1.0,H,...,5.0,12.0,10.0,12.0,3.0,6.0,0.0,3.0,1.0,0.0
2,E0,2002-08-17,Everton,Tottenham,2.0,2.0,D,1.0,0.0,H,...,9.0,5.0,18.0,4.0,10.0,5.0,1.0,1.0,0.0,0.0
3,E0,2002-08-17,Fulham,Bolton,4.0,1.0,H,3.0,1.0,H,...,6.0,1.0,16.0,12.0,7.0,4.0,1.0,2.0,0.0,0.0
4,E0,2002-08-17,Leeds,Man City,3.0,0.0,H,2.0,0.0,H,...,8.0,10.0,13.0,13.0,2.0,7.0,1.0,1.0,0.0,0.0


AIM: A function that takes as input the date, HomeTeam and AwayTeam. It will filter the df_epl_train dataframe for matches between HomeTeam and AwayTeam that took place before the input date. Then take an average of the 
columns like HR, AR, etc. This will provide us with the past stats for games played in past between the two teams. We can then use these past stats (between the two teams) as features to input into the classifier.

First, filter the dataframe to include only matches where date is less than date specified and also only include matches where HomeTeam=input(HomeTeam) and AwayTeam=input(AwayTeam):

In [170]:
# This function will take as input a date, HomeTeam and AwayTeam and output a filtered dataframe where the matches shown are played before input data and match is between HomeTeam and AwayTeam

# For Example:
# date = "24/06/2020"
# HomeTeam = "Newcastle"
# AwayTeam = "Aston Villa"

def get_season_start_date(date):
    if date.month <= 7:
        return datetime(date.year-3, 8, 1)
    return datetime(date.year-2, 8, 1)

def filter_dataframe_by_bothteams_history(df, date, HomeTeam, AwayTeam):
    # Convert the input string date into datetime
    date = pd.to_datetime(date, dayfirst=True)

    # Filter the dataframe to include only rows where Date<input(Date) && HomeTeam=input(HomeTeam) && AwayTeam=input(AwayTeam)
    df_filtered = df_epl_train.copy()
    df_filtered = df_filtered[(df_filtered.Date<date) & (df_filtered.HomeTeam==HomeTeam) & (df_filtered.AwayTeam==AwayTeam)]

    # Return filtered dataframe
    return df_filtered

def filter_dataframe_by_hometeam_recent_season(df, date, HomeTeam):
    # Convert the input string date into datetime
    date = pd.to_datetime(date, dayfirst=True)

    # Filter the dataframe to include only rows where Date<input(Date) && Date>input(first day of season) && HomeTeam=input(HomeTeam)
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered.Date<date) & (df_filtered.Date>get_season_start_date(date)) & (df_filtered.HomeTeam==HomeTeam)]

    # Return filtered dataframe
    return df_filtered

def filter_dataframe_by_awayteam_recent_season(df, date, AwayTeam):
    # Convert the input string date into datetime
    date = pd.to_datetime(date, dayfirst=True)

    # Filter the dataframe to include only rows where Date<input(Date) && Date>input(first day of season) && HomeTeam=input(HomeTeam)
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered.Date<date) & (df_filtered.Date>get_season_start_date(date)) & (df_filtered.AwayTeam==AwayTeam)]

    # Return filtered dataframe
    return df_filtered

# An example to see what the function does:
print(filter_dataframe_by_hometeam_recent_season(df_epl_train, "24/06/2020", "Newcastle"))

     Div       Date   HomeTeam          AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
5709  E0 2017-08-13  Newcastle         Tottenham   0.0   2.0   A   0.0   0.0   
5724  E0 2017-08-26  Newcastle          West Ham   3.0   0.0   H   1.0   0.0   
5744  E0 2017-09-16  Newcastle             Stoke   2.0   1.0   H   1.0   0.0   
5769  E0 2017-10-01  Newcastle         Liverpool   1.0   1.0   D   1.0   1.0   
5784  E0 2017-10-21  Newcastle    Crystal Palace   1.0   0.0   H   0.0   0.0   
5801  E0 2017-11-04  Newcastle       Bournemouth   0.0   1.0   A   0.0   0.0   
5824  E0 2017-11-25  Newcastle           Watford   0.0   3.0   A   0.0   2.0   
5853  E0 2017-12-09  Newcastle         Leicester   2.0   3.0   A   1.0   1.0   
5865  E0 2017-12-13  Newcastle           Everton   0.0   1.0   A   0.0   1.0   
5898  E0 2017-12-27  Newcastle          Man City   0.0   1.0   A   0.0   1.0   
5905  E0 2017-12-30  Newcastle          Brighton   0.0   0.0   D   0.0   0.0   
5923  E0 2018-01-13  Newcastle          

We now find the average of each of the columns that we need from this filtered dataframe e.g. HST, AST:

In [171]:
# This function takes as input the filtered dataframe from previous cell, features to average and a dictionary,
# it then appends an average of each feature to the dictionary

def average_columns(avg_features, filtered_df):
    for feature in avg_features.keys():
        df_col_means = filtered_df[feature].mean()
        avg_features[feature].append(df_col_means)

We now run the two functions on each row of the original dataframe to fill the dictionary with averages for each row

In [172]:
# Run the two functions for each row of our df_epl_train dataframe to fill dictionary with AVG for each match
# NOTE: Some matches won't have past stats since the two teams may not have played against each other in past or we might not have the data

# These are the features we want to get averages for both teams
# avg_features_both = {
#                     "FTHG": [],
#                     "FTAG": [],
#                     "HTHG": [],
#                     "HTAG": [],
#                     "HS"  : [],
#                     "AS"  : [],
#                     "HST" : [],
#                     "AST" : [],
#                     "HF"  : [],
#                     "AF"  : [],
#                     "HC"  : [],
#                     "AC"  : [],
#                     "HY"  : [],
#                     "AY"  : [],
#                     "HR"  : [],
#                     "AR"  : []
#                 }
avg_features_both = {
                    "HS"  : [],
                    "AS"  : [],
                    "HST" : [],
                    "AST" : [],
                    "HF"  : [],
                    "AF"  : [],
                    "HC"  : [],
                    "AC"  : [],
                    "HY"  : [],
                    "AY"  : [],
                    "HR"  : [],
                    "AR"  : []
                }

# These are the features we want to get averages for home team
# avg_features_home = {
#                     "FTHG": [],
#                     "HTHG": [],
#                     "HS"  : [],
#                     "HST" : [],
#                     "HF"  : [],
#                     "HC"  : [],
#                     "HY"  : [],
#                     "HR"  : [],
#                 }
avg_features_home = {
                    "HS"  : [],
                    "HST" : [],
                    "HF"  : [],
                    "HC"  : [],
                    "HY"  : [],
                    "HR"  : [],
                }

# These are the features we want to get averages for away team
# avg_features_away = {
#                     "FTAG": [],
#                     "HTAG": [],
#                     "AS"  : [],
#                     "AST" : [],
#                     "AF"  : [],
#                     "AC"  : [],
#                     "AY"  : [],
#                     "AR"  : []
#                   }
avg_features_away = {
                    "AS"  : [],
                    "AST" : [],
                    "AF"  : [],
                    "AC"  : [],
                    "AY"  : [],
                    "AR"  : []
                }


# Run the two functions on each row of the df_epl_train and fill the dictionary
# For each row in the dataframe
for index, row in df_epl_train.iterrows():
    # Filter the dataframe to only show matches played between those teams and before the certain date
    df_epl_train_average_bothteams_history = filter_dataframe_by_bothteams_history(df_epl_train, row["Date"],row["HomeTeam"],row["AwayTeam"])
    df_epl_train_average_hometeam_recent_season = filter_dataframe_by_hometeam_recent_season(df_epl_train, row["Date"],row["HomeTeam"])
    df_epl_train_average_awayteam_recent_season = filter_dataframe_by_awayteam_recent_season(df_epl_train, row["Date"],row["AwayTeam"])
    # Get averages from the filtered dataframe and add the the dictionary
    average_columns(avg_features_both, df_epl_train_average_bothteams_history)
    average_columns(avg_features_home, df_epl_train_average_hometeam_recent_season)
    average_columns(avg_features_away, df_epl_train_average_awayteam_recent_season)

In [173]:
print(avg_features_home)

{'HS': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 15.0, 5.0, 11.0, 15.0, 13.0, 13.0, 12.0, 15.0, 20.0, 6.0, 13.0, 8.0, 12.0, 8.0, 15.0, 11.0, 16.0, 20.0, 12.5, 6.5, 11.0, 11.0, 13.0, 19.0, 13.5, 13.0, 12.0, 8.5, 14.0, 7.5, 9.5, 12.0, 16.5, 12.5, 10.0, 13.5, 13.0, 12.0, 12.666666666666666, 16.666666666666668, 16.666666666666668, 7.333333333333333, 17.5, 11.666666666666666, 7.333333333333333, 11.333333333333334, 11.0, 13.0, 12.0, 11.666666666666666, 8.0, 8.666666666666666, 12.333333333333334, 13.666666666666666, 14.0, 11.0, 12.666666666666666, 10.333333333333334, 7.75, 15.333333333333334, 12.0, 7.75, 13.0, 11.5, 11.0, 11.333333333333334, 18.25, 16.5, 11.2, 13.0, 10.75, 12.5, 14.0, 10.25, 10.0, 7.75, 12.5, 12.4, 11.0, 12.2, 11.75, 9.5, 17.0, 16.6, 8.8, 15.0, 13.4, 8.8, 13.5, 11.2, 15.666666666666666, 16.333333333333332, 11.6, 12.833333333333334, 9.0, 10.166666666666666, 12.4, 11.2, 15.2, 13.333333333333334, 13.333333333333334, 13.6

Add these average feature lists in the dictionary back into the original dataframe as columns:

In [174]:
# Add a column for each of these feature averages using our list of values from the dictionary
df_epl_train_updated = df_epl_train.copy()

for feature in avg_features_both.keys():
    # Get the list of averages for a certain feature from the dicitonary
    feature_vals = avg_features_both[feature]
    # Add the list of averages into the dataframe for that certain feature
    df_epl_train_updated[feature + "_HISTORY"] = feature_vals

for feature in avg_features_home.keys():
    # Get the list of averages for a certain feature from the dicitonary
    feature_vals = avg_features_home[feature]
    # Add the list of averages into the dataframe for that certain feature
    df_epl_train_updated[feature + "_AVG"] = feature_vals
    
for feature in avg_features_away.keys():
    # Get the list of averages for a certain feature from the dicitonary
    feature_vals = avg_features_away[feature]
    # Add the list of averages into the dataframe for that certain feature
    df_epl_train_updated[feature + "_AVG"] = feature_vals

This is the new dataframe with the added columns with past average statistics for each row:

In [175]:
# Now this dataframe contains our original data + the average of the past stats for each row
print(df_epl_train_updated)

     Div       Date     HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
0     E0 2002-08-17    Blackburn      Sunderland   0.0   0.0   D   0.0   0.0   
1     E0 2002-08-17     Charlton         Chelsea   2.0   3.0   A   2.0   1.0   
2     E0 2002-08-17      Everton       Tottenham   2.0   2.0   D   1.0   0.0   
3     E0 2002-08-17       Fulham          Bolton   4.0   1.0   H   3.0   1.0   
4     E0 2002-08-17        Leeds        Man City   3.0   0.0   H   2.0   0.0   
...   ..        ...          ...             ...   ...   ...  ..   ...   ...   
7731  E0 2022-11-06      Chelsea         Arsenal   0.0   1.0   A   0.0   0.0   
7732  E0 2022-11-06  Aston Villa      Man United   3.0   1.0   H   2.0   1.0   
7733  E0 2022-11-06  Southampton       Newcastle   1.0   4.0   A   0.0   1.0   
7734  E0 2022-11-06     West Ham  Crystal Palace   1.0   2.0   A   1.0   1.0   
7735  E0 2022-11-06    Tottenham       Liverpool   1.0   2.0   A   0.0   2.0   

     HTR  ...     HF_AVG    HC_AVG    H

NOTE: Some rows have nan values, these are matches where teams might not have played each other before (or we dont have the past match data for them)

So we need to remove these rows (from the dataframe) where there are nan values. This is required to run the classifier training

In [176]:
# NOTE: We must remove the rows in the dataframe where the average values of stats/features are 'nan';
# we get these values because either the two teams have not played a match in the past OR because we have
# not got the past stats for these matches. We cannot use the 'nan' values for the classifier training and 
# hence have to remove these rows. We can then train a classifier using this final dataframe.

# In the final model/classifier, in the case where we DO NOT have these past stats of the two teams playing, 
# we need to switch back to using the OLD classifier which only took the 4 basic fetaures: day, month, 
# HomeTeam and AwayTeam.

# In the case where we DO have these past stats for two teams playing each other, we can use this model/classifier 
# and input the features like HST_AVG and AST_AVG. We would find these by using the filter_dataframe() and 
# average_columns() functions to find them for any two specific teams playing each other on some date.

# Remove any rows with nan
df_epl_train_final = df_epl_train_updated.dropna()
df_epl_train_final

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HF_AVG,HC_AVG,HY_AVG,HR_AVG,AS_AVG,AST_AVG,AF_AVG,AC_AVG,AY_AVG,AR_AVG
380,E0,2003-08-16,Arsenal,Everton,2.0,1.0,H,1.0,0.0,H,...,12.210526,8.210526,1.157895,0.052632,10.157895,5.473684,14.578947,5.578947,1.263158,0.157895
381,E0,2003-08-16,Birmingham,Tottenham,1.0,0.0,H,1.0,0.0,H,...,14.473684,5.157895,1.578947,0.052632,8.315789,4.684211,11.421053,5.157895,1.631579,0.157895
383,E0,2003-08-16,Fulham,Middlesbrough,3.0,2.0,H,1.0,1.0,D,...,11.842105,5.842105,1.052632,0.105263,8.157895,4.052632,12.368421,5.105263,1.526316,0.210526
385,E0,2003-08-16,Man United,Bolton,4.0,0.0,H,1.0,0.0,H,...,11.000000,8.421053,0.842105,0.000000,8.315789,3.842105,10.736842,4.052632,2.052632,0.105263
387,E0,2003-08-17,Charlton,Man City,0.0,3.0,A,0.0,2.0,A,...,12.368421,5.210526,1.263158,0.105263,9.473684,4.894737,12.684211,4.947368,1.842105,0.157895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7731,E0,2022-11-06,Chelsea,Arsenal,0.0,1.0,A,0.0,0.0,D,...,11.046512,6.627907,1.488372,0.069767,12.136364,4.113636,9.045455,4.500000,1.545455,0.136364
7732,E0,2022-11-06,Aston Villa,Man United,3.0,1.0,H,2.0,1.0,H,...,11.068182,5.431818,1.636364,0.113636,12.431818,5.045455,12.159091,4.659091,2.045455,0.022727
7733,E0,2022-11-06,Southampton,Newcastle,1.0,4.0,A,0.0,1.0,A,...,11.022727,5.477273,1.409091,0.045455,10.113636,3.636364,9.659091,4.500000,1.886364,0.022727
7734,E0,2022-11-06,West Ham,Crystal Palace,1.0,2.0,A,1.0,1.0,D,...,9.386364,4.818182,1.318182,0.022727,9.604651,3.581395,10.767442,4.093023,1.697674,0.069767


In [177]:
# Turn the catergorical data into labels using same method from before
df_epl_train_final["AwayTeam_Enc"] = df_epl_train_final["AwayTeam"].astype("category").cat.codes
df_epl_train_final["HomeTeam_Enc"] = df_epl_train_final["HomeTeam"].astype("category").cat.codes

# Transform the date column into day and month columns and Add into dataframe (Extract days & months from date)
df_epl_train_final["Date"] = pd.to_datetime(df_epl_train_final["Date"])
df_epl_train_final["Day"] = df_epl_train_final["Date"].dt.day
df_epl_train_final["Month"] = df_epl_train_final["Date"].dt.month 
df_epl_train_final["Year"] = df_epl_train_final["Date"].dt.year

# Check the final updated dataframe
df_epl_train_final

<ipython-input-177-d8d4dbd32394>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epl_train_final["AwayTeam_Enc"] = df_epl_train_final["AwayTeam"].astype("category").cat.codes
<ipython-input-177-d8d4dbd32394>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epl_train_final["HomeTeam_Enc"] = df_epl_train_final["HomeTeam"].astype("category").cat.codes
<ipython-input-177-d8d4dbd32394>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AST_AVG,AF_AVG,AC_AVG,AY_AVG,AR_AVG,AwayTeam_Enc,HomeTeam_Enc,Day,Month,Year
380,E0,2003-08-16,Arsenal,Everton,2.0,1.0,H,1.0,0.0,H,...,5.473684,14.578947,5.578947,1.263158,0.157895,13,0,16,8,2003
381,E0,2003-08-16,Birmingham,Tottenham,1.0,0.0,H,1.0,0.0,H,...,4.684211,11.421053,5.157895,1.631579,0.157895,33,2,16,8,2003
383,E0,2003-08-16,Fulham,Middlesbrough,3.0,2.0,H,1.0,1.0,D,...,4.052632,12.368421,5.105263,1.526316,0.210526,22,14,16,8,2003
385,E0,2003-08-16,Man United,Bolton,4.0,0.0,H,1.0,0.0,H,...,3.842105,10.736842,4.052632,2.052632,0.105263,4,21,16,8,2003
387,E0,2003-08-17,Charlton,Man City,0.0,3.0,A,0.0,2.0,A,...,4.894737,12.684211,4.947368,1.842105,0.157895,20,10,17,8,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7731,E0,2022-11-06,Chelsea,Arsenal,0.0,1.0,A,0.0,0.0,D,...,4.113636,9.045455,4.500000,1.545455,0.136364,0,11,6,11,2022
7732,E0,2022-11-06,Aston Villa,Man United,3.0,1.0,H,2.0,1.0,H,...,5.045455,12.159091,4.659091,2.045455,0.022727,21,1,6,11,2022
7733,E0,2022-11-06,Southampton,Newcastle,1.0,4.0,A,0.0,1.0,A,...,3.636364,9.659091,4.500000,1.886364,0.022727,23,29,6,11,2022
7734,E0,2022-11-06,West Ham,Crystal Palace,1.0,2.0,A,1.0,1.0,D,...,3.581395,10.767442,4.093023,1.697674,0.069767,12,36,6,11,2022


Create our X and y matrix and split into a training and test set:

In [178]:
# Create the input features matrix X (made of day, month, HomeTeam, AwayTeam, FTHG_AVG, FTAG_AVG, etc)
# Create the output values y vector (made of FTR)
# Take these values from the transformed dataframe

# X = df_epl_train_final.loc[:,['Day', 'Month', 'HomeTeam_Enc', 'AwayTeam_Enc','FTHG_AVG','FTAG_AVG','HTHG_AVG','HTAG_AVG','HS_AVG','AS_AVG','HST_AVG','AST_AVG','HF_AVG','AF_AVG','HC_AVG','AC_AVG','HY_AVG','AY_AVG','HR_AVG','AR_AVG']].values
X = df_epl_train_final.loc[:,['Day', 'Month', 'HomeTeam_Enc', 'AwayTeam_Enc',
                              'HS_HISTORY','AS_HISTORY','HST_HISTORY','AST_HISTORY','HF_HISTORY','AF_HISTORY','HC_HISTORY','AC_HISTORY','HY_HISTORY','AY_HISTORY','HR_HISTORY','AR_HISTORY',
                              'HS_AVG','AS_AVG','HST_AVG','AST_AVG','HF_AVG','AF_AVG','HC_AVG','AC_AVG','HY_AVG','AY_AVG','HR_AVG','AR_AVG']].values
y = df_epl_train_final.loc[:,'FTR'].values

# Split the training data in a 80-20 split
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=22)

# Encode the y output values as well
FTR_encoder = LabelEncoder()
y_train = FTR_encoder.fit_transform(y_train);

Now we can test using the different classifiers:

In [179]:
# Create an empty Tree model
DT_Model = DecisionTreeClassifier(random_state=42)
# Fit the model using training data
DT_Model.fit(X_train, y_train)
# Make predictions using the model we have created
DT_predictions_test = DT_Model.predict(X_test)
# Reconverting prediction values (i.e. 0, 1 or 2) back into (H, D or A) using the FTR_encoder defined in earlier cell
DT_predictions_test = FTR_encoder.inverse_transform(DT_predictions_test)

In [180]:
print(accuracy_score(DT_predictions_test, y_test))
print(classification_report(DT_predictions_test, y_test))

0.41185770750988143
              precision    recall  f1-score   support

           A       0.43      0.39      0.41       420
           D       0.25      0.31      0.28       285
           H       0.51      0.48      0.49       560

    accuracy                           0.41      1265
   macro avg       0.40      0.39      0.39      1265
weighted avg       0.42      0.41      0.42      1265



In [181]:
# Create an empty KNN model
KNN_Model = KNeighborsClassifier(n_neighbors=6)
# Fit the model using training data
KNN_Model.fit(X_train, y_train)
# Make predictions using the model we have created
KNN_predictions_test = KNN_Model.predict(X_test)
KNN_predictions_test = FTR_encoder.inverse_transform(KNN_predictions_test)

In [182]:
print(accuracy_score(KNN_predictions_test, y_test))
print(classification_report(KNN_predictions_test, y_test))

0.44110671936758894
              precision    recall  f1-score   support

           A       0.50      0.43      0.46       457
           D       0.16      0.29      0.20       190
           H       0.58      0.50      0.54       618

    accuracy                           0.44      1265
   macro avg       0.42      0.40      0.40      1265
weighted avg       0.49      0.44      0.46      1265



In [183]:
# Create an empty Random Forest model
RF_Model = RandomForestClassifier(n_estimators=50, random_state=42)
# Fit the model using training data
RF_Model.fit(X_train, y_train)
# Make predictions using the model we have created
RF_predictions_test = RF_Model.predict(X_test)
RF_predictions_test = FTR_encoder.inverse_transform(RF_predictions_test)

In [184]:
print(accuracy_score(RF_predictions_test, y_test))
print(classification_report(RF_predictions_test, y_test))

0.5011857707509881
              precision    recall  f1-score   support

           A       0.51      0.49      0.50       398
           D       0.11      0.37      0.16       101
           H       0.76      0.52      0.62       766

    accuracy                           0.50      1265
   macro avg       0.46      0.46      0.43      1265
weighted avg       0.63      0.50      0.55      1265

